In [1]:
# !/root/anaconda3/bin/pip install psycopg2-binary

## 連線postgresql的套件
import psycopg2
## 建立資料結構的套件，使用於資料處理，如：分割、合併等等 (Python data analysis)
import pandas as pd

## 設定連接資料庫參數
conn = psycopg2.connect(database="mimic", user="twmimicviewer", 
                        password="twmimicviewer@1234", host="203.145.218.182", port="5432")

print(conn.closed)


0


# 參數介紹
* database 資料庫名稱
* user 帳號
* password 密碼
* host 伺服器IP
* port 連接資料庫的埠

In [7]:
## 連接上資料庫，抓取資料。

## 允許python執行postgresql語法
with conn.cursor() as cur:

    ## 觀察查詢筆數
    print(cur.rowcount)
    
    ## SQL語法
    sql = "select * from mimiciii.icustays"
    
    ## 執行sql語法
    cur.execute(sql)
    
    ## 取得欄位名稱
    print(cur.description)
    name = [desc[0] for desc in cur.description]
    
    ## fetchall是將所有收尋的資料都寫入 rows 中
    # rows = pd.DataFrame(cur.fetchall())
    rows = pd.DataFrame(cur.fetchall(),columns=name)
    
    ## 取出一筆資料
    # rows = pd.DataFrame(cur.fetchone())
    
    ## 自行設定要幾筆資料
    # rows = pd.DataFrame(cur.fetchmany(10))
    
## 觀察查詢筆數
print(cur.rowcount)

-1
(Column(name='row_id', type_code=23), Column(name='subject_id', type_code=23), Column(name='hadm_id', type_code=23), Column(name='icustay_id', type_code=23), Column(name='dbsource', type_code=1043), Column(name='first_careunit', type_code=1043), Column(name='last_careunit', type_code=1043), Column(name='first_wardid', type_code=21), Column(name='last_wardid', type_code=21), Column(name='intime', type_code=1114), Column(name='outtime', type_code=1114), Column(name='los', type_code=701))
61532


In [50]:
print(rows.shape)

rows.head()

(61532, 12)


,row_id,subject_id,hadm_id,icustay_id,dbsource,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


# 合併與整理兩個資料表，拿取資料，以敗血病為例

In [8]:
## 改寫SQL語法

conn = psycopg2.connect(database="mimic", user="twmimicviewer", 
                        password="twmimicviewer@1234", host="203.145.218.182", port="5432")

with conn.cursor() as cur:

    ## 觀察查詢筆數
    
    ## SQL語法
    sql = """
    select ma.* 
    from mimiciii.admissions ma
    INNER JOIN public.angus pa
    ON ma.hadm_id = pa.hadm_id
    where pa.explicit_sepsis = 1
    """
    
    ## 執行sql語法
    cur.execute(sql)
    
    ## 取得欄位名稱
    name = [desc[0] for desc in cur.description]
    
    ## 取得資料
    rows = pd.DataFrame(cur.fetchall(),columns=name)
    

In [9]:
print(rows.shape)
rows.head()

(4085, 19)


,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,40,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,None,CATHOLIC,WIDOWED,WHITE,NaT,NaT,ACUTE MYOCARDIAL INFARCTION-SEPSIS,0,1
1,457,357,145674,2198-08-02 04:49:00,2198-10-26 14:55:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-08-02 02:03:00,2198-08-02 05:37:00,SEIZURE,0,1
2,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-11-01 18:01:00,2198-11-01 23:06:00,SEPSIS,0,1
3,169,124,138376,2165-12-31 18:55:00,2166-02-01 06:55:00,2166-02-01 06:55:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,PNEUMONIA;CHRONIC OBST PULM DISEASE,1,1
4,152,111,155897,2144-07-01 04:12:00,2144-07-01 14:55:00,2144-07-01 14:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,PROTESTANT QUAKER,MARRIED,WHITE,2144-07-01 02:30:00,2144-07-01 11:21:00,PNEUMONIA,1,1
